[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1Qo2KBymNdJq1A9t22oeNDpeim7YYhmqY)

#### Importing packages that will be used.

In [ ]:
import numpy as np
from scipy import special
import random
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import pandas as pd
import matplotlib.pyplot as plt

#### Specify the number of parameters to sample and the number of samples to draw from each parameter distribution.

In [ ]:
# Number of parameters to sample 
parameterCount = 5;
# Number of samples to draw for each parameter
sampleCount = 500; 

#### This defines functions for specifying sampled parameters' names and distributions  as well as drawing samples from a user-specified parameter distribution. Does not need any user edits.

In [ ]:
def parNameDist(Name,Distribution):
    paramTemp = {}
    paramTemp['Name']=Name
    paramTemp['Dist']=Distribution
    
    return paramTemp

def sampleDistrib(modelParamName,distrib,distribSpecs): 
    
    if distrib == 'uniform':
        
        mmin = distribSpecs[0].value
        mmax = distribSpecs[1].value
        
        intervalwidth = (mmax - mmin) / sampleCount      # width of each 
                                                         # sampling interval
        samples = []
        
        for sample in range(sampleCount):
            
            lower = mmin + intervalwidth * (sample)    # lb of interval
            upper = mmin + intervalwidth * (sample+1)      # ub of interval
            
            sampleVal = np.random.uniform(lower, upper)  # draw a random sample 
                                                         # within the interval
            samples.append(sampleVal)

    
    
    elif distrib == 'normal':
        
        mmean= distribSpecs[0].value
        mvar = distribSpecs[1].value
        
        lower = mvar*np.sqrt(2)*special.erfinv(-0.9999)+mmean # set lb of 1st
                                                              # sample interval
        samples = []
        
        for sample in range(sampleCount):
          
            n = sample + 1
            
            if n != sampleCount:
                upper = (np.sqrt(2*mvar)*special.erfinv(2*n/sampleCount-1)
                         + mmean)                        # ub of sample interval
            else:
                upper = np.sqrt(2*mvar)*special.erfinv(0.9999) + mmean
 
            sampleVal = np.random.uniform(lower, upper)  # draw a random sample 
                                                         # within the interval
    
            samples.append(sampleVal)

            lower = upper           # set current ub as the lb for next interval
            

    
    elif distrib == 'triangle':
        
        mmin = distribSpecs[0].value
        mmax = distribSpecs[1].value
        mmode= distribSpecs[2].value
    
        samples = []
        
        for sample in range(sampleCount):
          
            n = sample + 1
            
            intervalarea = 1/sampleCount 
            
            ylower = intervalarea*(n-1) # use cdf to read off area as y's &
            yupper = intervalarea*(n)   # get corresponding x's for the pdf
        
        
            # Check to see if y values = cdf(x <= mmode) 
            # for calculating correxponding x values:
            
            if ylower <= ((mmode - mmin)/(mmax - mmin)):     
                lower = np.sqrt(ylower*(mmax - mmin)*(mmode - mmin)) + mmin 

            else:
                lower = mmax-np.sqrt((1 - ylower)*(mmax - mmin)*(mmax - mmode))

            
            if yupper <= ((mmode - mmin)/(mmax - mmin)):    
                upper = np.sqrt(yupper*(mmax - mmin)*(mmode - mmin)) + mmin; 

            else:
                upper = mmax-np.sqrt((1 - yupper)*(mmax - mmin)*(mmax - mmode))

                
            sampleVal = np.random.uniform(lower, upper)  
            
            samples.append(sampleVal)
            
    
    b = int(np.ceil(sampleCount))
    plt.hist(samples, density = 0, bins = b, histtype = 'step') 
    
    B=str(b)
    
    plt.title('Histogram of ' + modelParamName 
              + ' parameter samples for ' + B + ' bins')
    
    plt.ylabel('proportion of samples');
    plt.xlabel(modelParamName + ' value')
    
    plt.show()
    
    return samples

#### Calls the function to ask for user input to name parameters and specify distributions. Type these in text input boxes and dropdowns that will appear below after running the cell.

In [ ]:
params = {}

for i in range(parameterCount):
  
    s=str(i)
    
    params[i] = interactive(parNameDist,
                            Name='Type parameter ' + s + ' name', 
                            Distribution=['uniform','normal','triangle'])
    
    display(params[i])

#### Input parameter distribution specifics in the interactive boxes that appear below after running this cell.

In [ ]:
distribSpecs={}

for i in range(parameterCount):
  
    parName = params[i].result['Name']
    
    print('Enter distribution specifics for parameter ' + parName + ':')
    
    if params[i].result['Dist'] == 'normal':

        distribSpecs[parName] = {}
        
        distribSpecs[parName][0] = widgets.FloatText(
                value=2,
                description='Mean:'
              )
        distribSpecs[parName][1] = widgets.FloatText(
                value=1,
                description='Variance:'
              )

        display(distribSpecs[parName][0], distribSpecs[parName][1])

    elif params[i].result['Dist'] == 'uniform':

        distribSpecs[parName] = {}

        distribSpecs[parName][0] = widgets.FloatText(
                value=0,
                description='Minimum:'
              )
        distribSpecs[parName][1] = widgets.FloatText(
                value=2,
                description='Maximum:'
              )

        display(distribSpecs[parName][0], distribSpecs[parName][1])


    elif params[i].result['Dist'] == 'triangle':
      
        distribSpecs[parName] = {}

        distribSpecs[parName][0] = widgets.FloatText(
                value=0,
                description='Minimum:'
              )
        distribSpecs[parName][1] = widgets.FloatText(
                value=2,
                description='Maximum:'
              )
        distribSpecs[parName][2] = widgets.FloatText(
                value=1,
                description='Mode:'
              )

        display(distribSpecs[parName][0], distribSpecs[parName][1], distribSpecs[parName][2])

#### This passes the distributions to the code for generating parameter samples, and histogram plots of samples for each parameter will appear below.

In [ ]:
parameters = {}
for j in range(parameterCount):
  
    parameters[params[j].result['Name']] = sampleDistrib(params[j].result['Name'],
                                                         params[j].result['Dist'],
                                                         distribSpecs[params[j].result['Name']])

#### Randomly permute each set of parameter samples in order to randomly pair the samples to more fully sample the parameter space for the Monte Carlo simulations.

In [ ]:
LHSparams=[]

for p in parameters:
    temp = parameters[p]
    random.shuffle(temp)
    
    LHSparams.append(temp)

#### Definition of the model function.

In [ ]:
from scipy.sparse import diags #linalg
from scipy.linalg import solve
import numpy as np
from numpy.linalg import inv, det
from matplotlib import pyplot as plt

In [ ]:
np.seterr( over ='ignore')

In [ ]:
a1 = 0 
b1 = 7200
c1 = 2*b1
d1 = 3*b1
e1 = 4*b1
f1 = 5*b1
g1 = 6*b1
h1 = 7*b1
i1 = 8*b1
j1 = 9*b1
k1 = 10*b1
l1 = 11*b1
m1 = 12*b1
n1 = 13*b1
o1 = 14*b1
p1 = 15*b1
q1 = 16*b1
r1 = 17*b1

z = 0.01

In [ ]:
#Depth per month of the sources
J1 = 3.93
J2 = 3.93
J3 = 0.73
J4 = 0.63
J5 = 0.8
J6 = 0.9

F1 = 4.03
F2 = 3.57
F3 = 0.53
F4 = 0.6
F5 = 1.17
F6 = 0.8

Mar1 = 3.83
Mar2 = 3.53 
Mar3 = 1.07
Mar4 = 0.37
Mar5 = 1.23
Mar6 = 0.67

Ap1 = 4.03
Ap2 = 3.97
Ap3 = 0.77
Ap4 = 0.53
Ap5 = 0.9
Ap6 = 0.57

Ma1 = 4.43
Ma2 = 3.4
Ma3 = 0.63
Ma4 = 0.9
Ma5 = 1.03
Ma6 = 0.63

Jun1 = 3.75
Jun2 = 3.8
Jun3 = 1.4
Jun4 = 0.7
Jun5 = 1.67
Jun6 = 1.03

Jul1 = 4.27
Jul2 = 4.23
Jul3 = 1.1
Jul4 = 1.15
Jul5 = 1.17
Jul6 = 1.2

Au1 = 4.03
Au2 = 4.53
Au3 = 1.73
Au4 = 1.03
Au5 = 1.3
Au6 = 1.73

S1 = 3.43
S2 = 3.9
S3 = 1.5
S4 = 0.77
S5 = 1.67
S6 = 1.73

O1 = 3.97
O2 = 3.6
O3 = 0.87
O4 = 0.6
O5 = 1.5
O6 = 0.77

N1 = 3.05
N2 = 3.3
N3 = 0.77
N4 = 0.47
N5 = 0.97
N6 = 0.8

D1 = 2.87
D2 = 2.43
D3 = 0.67
D4 = 0.43
D5 = 1.43
D6 = 0.8

In [ ]:
#Wetted Perimeter
PJ1 = 2*J1 + 65
PJ2 = 2*J2 + 53
PJ3 = 2*J3 + 7
PJ4 = 2*J4 + 7
PJ5 = 2*J5 + 9
PJ6 = 2*J6 + 7

PF1 = 2*F1 + 65
PF2 = 2*F2 + 53
PF3 = 2*F3 + 7
PF4 = 2*F4 + 7
PF5 = 2*F5 + 9
PF6 = 2*F6 + 7

PMar1 = 2*Mar1 +  65
PMar2 = 2*Mar2 +  53
PMar3 = 2*Mar3 +  7
PMar4 = 2*Mar4 +  7
PMar5 = 2*Mar5 +  9
PMar6 = 2*Mar6 +  7

PAp1 = 2*Ap1 + 65
PAp2 = 2*Ap2 + 53
PAp3 = 2*Ap3 + 7
PAp4 = 2*Ap4 + 7
PAp5 = 2*Ap5 + 9
PAp6 = 2*Ap6 + 7

PMa1 = 2*Ma1 + 65
PMa2 = 2*Ma2 + 53
PMa3 = 2*Ma3 + 7
PMa4 = 2*Ma4 + 7
PMa5 = 2*Ma5 + 9
PMa6 = 2*Ma6 + 7

PJun1 = 2*Jun1 + 65
PJun2 = 2*Jun2 + 53
PJun3 = 2*Jun3 + 7
PJun4 = 2*Jun4 + 7
PJun5 = 2*Jun5 + 9
PJun6 = 2*Jun6 + 7

PJul1 = 2*Jul1 + 65
PJul2 = 2*Jul2 + 53
PJul3 = 2*Jul3 + 7
PJul4 = 2*Jul4 + 7
PJul5 = 2*Jul5 + 9
PJul6 = 2*Jul6 + 7

PAu1 = 2*Au1 + 65
PAu2 = 2*Au2 + 53
PAu3 = 2*Au3 + 7
PAu4 = 2*Au4 + 7
PAu5 = 2*Au5 + 9
PAu6 = 2*Au6 + 7

PS1 = 2*S1 + 65
PS2 = 2*S2 + 53
PS3 = 2*S3 + 7
PS4 = 2*S4 + 7
PS5 = 2*S5 + 9
PS6 = 2*S6 + 7

PO1 = 2*O1 + 65
PO2 = 2*O2 + 53
PO3 = 2*O3 + 7
PO4 = 2*O4 + 7
PO5 = 2*O5 + 9
PO6 = 2*O6 + 7

PN1 = 2*N1 + 65
PN2 = 2*N2 + 53
PN3 = 2*N3 + 7
PN4 = 2*N4 + 7
PN5 = 2*N5 + 9
PN6 = 2*N6 + 7

PD1 = 2*D1 + 65
PD2 = 2*D2 + 53
PD3 = 2*D3 + 7
PD4 = 2*D4 + 7
PD5 = 2*D5 + 9
PD6 = 2*D6 + 7

In [ ]:
#depth and wetted perimeter of Guadalope and Lambingan
DepthBamGuadJ = 3.92
DepthBamGuadF = 3.48
DepthBamGuadMar = 3.55
DepthBamGuadAp = 3.98
DepthBamGuadMa = 3.73
DepthBamGuadJun = 3.39
DepthBamGuadJul = 3.74
DepthBamGuadAu = 3.6
DepthBamGuadS = 3.28
DepthBamGuadO = 2.85
DepthBamGuadN = 3.14
DepthBamGuadD = 2.52


DepthGuadJ = 3.9
DepthGuadF = 2.93
DepthGuadMar = 3.27
DepthGuadAp = 3.93
DepthGuadMa = 3.03
DepthGuadJun = 3.03
DepthGuadJul = 3.2
DepthGuadAu = 3.17
DepthGuadS = 3.13
DepthGuadO = 1.73
DepthGuadN = 3.23
DepthGuadD = 2.17

DepthGuadLamJ = 3.40
DepthGuadLamF = 3.27
DepthGuadLamMar = 3.55
DepthGuadLamAp = 4.18
DepthGuadLamMa = 3.42
DepthGuadLamJun = 3.57
DepthGuadLamJul = 3.92
DepthGuadLamAu = 3.27
DepthGuadLamS = 3.10
DepthGuadLamO = 1.78
DepthGuadLamN = 3.22
DepthGuadLamD = 2.82

DepthLamJ = 2.9
DepthLamF = 3.6
DepthLamMar = 3.83
DepthLamAp = 4.43 
DepthLamMa = 3.8
DepthLamJun = 4.1
DepthLamJul = 4.63
DepthLamAu = 3.37
DepthLamS = 3.07
DepthLamO = 1.83
DepthLamN = 3.2
DepthLamD = 3.47

PeriBamGuadJ = 2*DepthBamGuadJ + 70
PeriBamGuadF = 2*DepthBamGuadF + 70
PeriBamGuadMar = 2*DepthBamGuadMar + 70
PeriBamGuadAp = 2*DepthBamGuadAp + 70
PeriBamGuadMa = 2*DepthBamGuadMa + 70
PeriBamGuadJun = 2*DepthBamGuadJun + 70
PeriBamGuadJul = 2*DepthBamGuadJul + 70
PeriBamGuadAu = 2*DepthBamGuadAu + 70
PeriBamGuadS = 2*DepthBamGuadS + 70
PeriBamGuadO = 2*DepthBamGuadO + 70
PeriBamGuadN = 2*DepthBamGuadN + 70
PeriBamGuadD = 2*DepthBamGuadD + 70

PeriGuadJ = 2*DepthGuadJ + 76
PeriGuadF =  2*DepthGuadF + 76
PeriGuadMar =  2*DepthGuadMar + 76
PeriGuadAp =  2*DepthGuadAp + 76
PeriGuadMa =  2*DepthGuadMa + 76
PeriGuadJun =  2*DepthGuadJun + 76
PeriGuadJul =  2*DepthGuadJul + 76
PeriGuadAu =  2*DepthGuadAu + 76
PeriGuadS =  2*DepthGuadS + 76
PeriGuadO =  2*DepthGuadO + 76
PeriGuadN =  2*DepthGuadN + 76
PeriGuadD =  2*DepthGuadD + 76

PeriGuadLamJ = 2*DepthGuadLamJ + 80
PeriGuadLamF = 2*DepthGuadLamF + 80
PeriGuadLamMar = 2*DepthGuadLamMar + 80
PeriGuadLamAp = 2*DepthGuadLamAp + 80
PeriGuadLamMa = 2*DepthGuadLamMa + 80
PeriGuadLamJun = 2*DepthGuadLamJun + 80
PeriGuadLamJul = 2*DepthGuadLamJul + 80
PeriGuadLamAu = 2*DepthGuadLamAu + 80
PeriGuadLamS = 2*DepthGuadLamS + 80
PeriGuadLamO = 2*DepthGuadLamO + 80
PeriGuadLamN = 2*DepthGuadLamN + 80
PeriGuadLamD = 2*DepthGuadLamD + 80

PeriLamJ = 2*DepthLamJ + 85
PeriLamF = 2*DepthLamF + 85
PeriLamMar = 2*DepthLamMar + 85
PeriLamAp = 2*DepthLamAp + 85
PeriLamMa = 2*DepthLamMa + 85
PeriLamJun = 2*DepthLamJun + 85
PeriLamJul = 2*DepthLamJul + 85
PeriLamAu = 2*DepthLamAu + 85
PeriLamS = 2*DepthLamS + 85
PeriLamO = 2*DepthLamO + 85
PeriLamN = 2*DepthLamN + 85
PeriLamD = 2*DepthLamD + 85

hb = 1

In [ ]:
def solver(T, M, sampledParams): # (y, t, sampledParams, unsampledParams):
    N = int(round(T/dt))
    t = np.linspace(0,T,N+1) #time discrete
    M = int(round(L/dx))
    x = np.linspace(0,L,M+1) #space discrete
    E = np.zeros((M+1,N+1))
    S = np.zeros((M+1,N+1))
    C = np.zeros((M+1,N+1))
    PPX = np.zeros((M+1))
    Em_E = np.zeros((M+1,N+1))
    BM = np.zeros((M+1,12))
    Bi = np.zeros((M+1))
    RHS = np.zeros((M+1))
    B = np.zeros((M+1))
    A = np.zeros((M+1))
    QW = np.zeros((M+1,12))
    Q = np.zeros((M+1,N+1))
    
    K, Q1, Q2, Q3, dummy = sampledParams
    
    #initial conditions
    S[0,:] = 0.27
    C[0,:] = z*S[0,:]
    E[0,:] = PJ1*den*hb*S[0,:]
    
     #Flow Rate Boundary
    QW[0,0] = Q1
    QW[24,0] = 18
    QW[66,0] = 2.04
    QW[72,0] = 0
    QW[88,0] = 0.65
    QW[148,0] = 0.88
    
    #boundary/sources
    BM[0,0] = QW[0,0]*(PJ1*den*hb*0.27)  
    BM[24,0] = QW[24,0]*(PJ2*den*J2*0.30)
    BM[66,0] = QW[66,0]*(PJ3*den*J3*0.44)
    BM[72,0] = QW[72,0]*(PJ4*den*J4*0.44)
    BM[88,0] = QW[88,0]*(PJ5*den*J5*0.41)
    BM[148,0] = QW[148,0]*(PJ6*den*J6*0.38)
   
    Em_E[:,:] = 0.99009901
    
    K = sampledParams[0]
    Q1 = sampledParams[1]
    Q2 = sampledParams[2]
    Q3 = sampledParams[3]
    dummy = sampledParams[4]

    
    Q[a2:b2,1] = Q1
    Q[b2:c2,1] = (Q1+Q2)/2
    Q[c2:d2,1] = Q2
    Q[d2:e2,1] = Q2
    Q[e2:f2,1] = (Q2+Q3)/2
    Q[f2:g2,1] = Q3
    
    #special time step at n=0
    A = A1[:,0]
    B = BM[:,0]
    L222 = np.multiply(L2,A)
    L333 = np.multiply(L3,A)
    Z1 = np.multiply(Z,A)
    L11 = np.multiply(L1,Q[0:M+1,1])
    L111 = np.multiply(L11,Em_E[0:M+1,0])
    PPX[:] = np.dot(QP, Em_E[0:M+1,0])
    PPX[PPX<0]=0 
    L22 = np.multiply(L222,PPX[:])
    L33 = np.multiply(L333, Em_E[0:M+1,0])
    M1 = Z1 / dt +  (L111 + L22*K + L33*K)
    E[:,1] = np.linalg.solve(M1, B)
    S[0,1] = S[0,a1]
    S[1:b2,1] = E[1:b2,1] / (den*PJ1*hb+z*den*PJ1*hb*theta)
    S[b2:c2,1] = E[b2:c2,1] / (den*PeriBamGuadJ*hb+z*den*PeriBamGuadJ*hb*theta)
    S[c2:d2,1] = E[c2:d2,1] / (den*PeriGuadJ*hb+z*den*PeriGuadJ*hb*theta)
    S[d2:e2,1] = E[d2:e2,1] / (den*PeriGuadJ*hb+z*den*PeriGuadJ*hb*theta)
    S[e2:f2,1] = E[e2:f2,1] / (den*PeriGuadLamJ*hb+z*den*PeriGuadLamJ*hb*theta)
    S[f2:g2,1] = E[f2:g2,1] / (den*PeriLamJ*hb+z*den*PeriLamJ*hb*theta)
    S[g2:h2,1] = E[g2:h2,1] / (den*PeriLamJ*hb+z*den*PeriLamJ*hb*theta)
    C[:,1] = z * S[:,1]
    
    
    for n in range(1,N):
        #if n<=b1: #January
        A = A1[:,0]
        L222 = np.multiply(L2,A)
        L333 = np.multiply(L3,A)
        Z1 = np.multiply(Z,A)
        B = BM[:,0]
        L11 = np.multiply(L1,Q[0:M+1,1])
        L111 = np.multiply(L11,Em_E[0:M+1,n])
        PPX[:] = np.dot(QP, Em_E[0:M+1,n])
        PPX[PPX<0]=0 
        L22 = np.multiply(L222,PPX[:])
        L33 = np.multiply(L333, Em_E[0:M+1,n])
        M1 = Z1 / dt + (L111 + L22*K + L33*K)
        M2 = Z1 / dt - (L111 + L22*K + L33*K)
        RHS[:] = M2.dot(E[:,n]) + B
        RHS[0] = PJ1*den*hb*0.27
        RHS[M] = -Q3*E[M,n]
        M1[0,0] = 1
        M1[0,1] = 0
        E[:,n+1] = np.linalg.solve(M1, RHS)
        S[0,n+1] = E[0,n+1] / (den*PJ1*hb)
        S[1:b2,n+1] = E[1:b2,n+1] / (den*PJ1*hb+z*den*PJ1*hb*theta)
        S[b2:c2,n+1] = E[b2:c2,n+1] / (den*PeriBamGuadJ*hb+z*den*PeriBamGuadJ*hb*theta)
        S[c2:d2,n+1] = E[c2:d2,n+1] / (den*PeriGuadJ*hb+z*den*PeriGuadJ*hb*theta)
        S[d2:e2,n+1] = E[d2:e2,n+1] / (den*PeriGuadJ*hb+z*den*PeriGuadJ*hb*theta)
        S[e2:f2,n+1] = E[e2:f2,n+1] / (den*PeriGuadLamJ*hb+z*den*PeriGuadLamJ*hb*theta)
        S[f2:g2,n+1] = E[f2:g2,n+1] / (den*PeriLamJ*hb+z*den*PeriLamJ*hb*theta)
        S[g2:h2,n+1] = E[g2:h2,n+1] / (den*PeriLamJ*hb+z*den*PeriLamJ*hb*theta)
        C[:,n+1] = z * S[:,n+1]
            
        
            
    return   S, x, t



In [ ]:
a2 = 0
b2 = 20
c2 = 58
d2 = 78
e2 = 101
f2 = 147
g2 = 171
h2 = 181

In [ ]:
A1 = np.zeros((M+1,12))
A = np.zeros((M+1))

A1[0:20,0] = 255
A1[20:58,0] = 276
A1[58:78,0] = 296
A1[78:101,0] = 296
A1[101:147,0] = 271
A1[147:171,0] = 247


In [ ]:
from scipy.integrate import quad
#Solving for matrices L1, L2, L3
T =  2592000
L = 8500
den = 0.01       #density of CIMW and CMW
theta = 1       #porosity
l = 50          #element distance
#K = 1000         #dispersion coefficient K=diffusion+dispersitivity*fv
dt = 360        #time step size
dx = 50
N = int(round(T/dt))          #no. of time steps
M = int(round(L/dx))          #no. of nodes

#Solving for L1, let E=E^m/E (to be solve)
def integrand(x):
    return -(-1/l)*(1-x/l)
ans1, err = quad(integrand, 0, l)

def integrand(x):
    return -(-1/l)*(x/l)
ans2, err = quad(integrand, 0, l)

def integrand(x):
    return -(1/l)*(1-x/l)
ans3, err = quad(integrand, 0, l)

def integrand(x):
    return -(1/l)*(x/l)
ans4, err = quad(integrand, 0, l)

L1 = diags([ans3,(ans1+ans4),ans2], [-1,0,1], shape=(M+1, M+1)).toarray()
L1[0, 0] = ans1
L1[M, M] = ans4

#Solving QP1

def integrand(x):
    return (1-x/l)*(1-x/l)
ans5, err = quad(integrand, 0, l)

def integrand(x):
    return (1-x/l)*(x/l)
ans6, err = quad(integrand, 0, l)

def integrand(x):
    return (x/l)*(x/l)
ans7, err = quad(integrand, 0, l)

QP1_11 = ans5
QP1_12 = ans6
QP1_21 = ans6
QP1_22 = ans7

#QP1 Lump Matrix
QP11 = ans5+ans6
QP22 = (ans5+2*ans6+ans7)*np.ones(M+1)
QP21 = ans6+ans7
QP1 = np.zeros((M+1)**2).reshape(M+1, M+1)+np.diag(QP22, 0)
QP1[0, 0] = QP11
QP1[M, M] = QP21

#Solving for QP2

def integrand(x):
    return (1-x/l)*(-1/l)
ans8, err = quad(integrand, 0, l)

def integrand(x):
    return (1-x/l)*(1/l)
ans9, err = quad(integrand, 0, l)

def integrand(x):
    return (x/l)*(-1/l)
ans10, err = quad(integrand, 0, l)

def integrand(x):
    return (x/l)*(1/l)
ans11, err = quad(integrand, 0, l)

QP2_11 = ans8
QP2_12 = ans9
QP2_21 = ans10
QP2_22 = ans11

#Matrix QP
QP_11 = QP2_11/(ans5+2*ans6+ans7)
QP_12 = QP2_12/(ans5+2*ans6+ans7)
QP_21 = QP2_21/(ans5+2*ans6+ans7)
QP_22 = QP2_22/(ans5+2*ans6+ans7)
qp1 = (QP_11+QP_22)*np.ones(M+1)
qp2 = (QP_12)*np.ones(M)
qp3 = (QP_21)*np.ones(M)
QP = np.zeros((M+1)**2).reshape(M+1,M+1)+np.diag(qp1,0)+np.diag(qp2,1)++np.diag(qp3,-1)
QP[0,0] = QP2_11/QP11
QP[0,1] = QP2_12/QP11
QP[M,M] = QP2_21/QP21
QP[M,M] = QP2_22/QP21


#Solving for L2

def integrand(x):
    return (-1/l)*(1-x/l)
ans12, err = quad(integrand, 0, l)

def integrand(x):
    return (-1/l)*(x/l)
ans13, err = quad(integrand, 0, l)

def integrand(x):
    return (1/l)*(1-x/l)
ans14, err = quad(integrand, 0, l)

def integrand(x):
    return (1/l)*(x/l)
ans15, err = quad(integrand, 0, l)

L2 = diags([ans14,(ans12+ans15),ans13], [-1,0,1], shape=(M+1, M+1)).toarray()
L2[0,0] = ans12
L2[M,M] = ans15

#Solving for L3

def integrand(x):
    return (-1/l)*(-1/l)
ans16, err = quad(integrand, 0, l)

def integrand(x):
    return (-1/l)*(1/l)
ans17, err = quad(integrand, 0, l)

def integrand(x):
    return (1/l)*(-1/l)
ans18, err = quad(integrand, 0, l)

def integrand(x):
    return (1/l)*(1/l)
ans19, err = quad(integrand, 0, l)

L3 = diags([ans18,(ans16+ans19),ans17], [-1,0,1], shape=(M+1, M+1)).toarray()
L3[0,0] = ans16
L3[M,M] = ans19


#Solving for M

def integrand(x):
    return (1-x/l)*(1-x/l)
ans20, err = quad(integrand, 0, l)

def integrand(x):
    return (1-x/l)*(x/l)
ans21, err = quad(integrand, 0, l)

def integrand(x):
    return (x/l)*(1-x/l)
ans22, err = quad(integrand, 0, l)

def integrand(x):
    return (x/l)*(x/l)
ans23, err = quad(integrand, 0, l)

Z = diags([ans22,(ans20+ans23),ans21], [-1,0,1], shape=(M+1, M+1)).toarray()
Z[0, 0] = ans20
Z[M, M] = ans23

#(S,x,t) = solver(T,dt,L1, L2, L3, Z, K, D, F, G)

#### Run Monte Carlo simulations for each parameter sample set.

In [ ]:
import scipy.integrate as spi

Simdata={}

Output = [] 

for i in range(sampleCount):
  
     Simdata[i]={}
    
for j in range(sampleCount):
    
    sampledParams=[i[j] for i in LHSparams]
    S,x,t=solver(T, M, sampledParams)
    Simdata[j] = S # solution S to the equation for variable
     
    
    Output.append(S)                                           

#### Compute partial rank correlation coefficients to compare simulation outputs with parameters

In [ ]:
SampleResult=[]

x_idx = 72          # time or location index of sim results 
x_idx2= x_idx+1     #   to compare w/ param sample vals
x_idt = 100

LHS=[*zip(*LHSparams)]
LHSarray=np.array(LHS)
#LHSarray = np.delete(LHSarray, (12,25,30,44,54,71,82), axis = 0)
Outputarray=np.array(Output)
subOut=Outputarray[0:,x_idx:x_idx2, x_idt]
#subOut = np.delete(subOut, (12,25,30,44,54,71,82), axis = 0)

LHSout = np.hstack((LHSarray,subOut))
SampleResult = LHSout.tolist()


Ranks2=[]              
for s in range(parameterCount):

    indices = list(range(len(LHSarray[:,s])))
    indices.sort(key=lambda k: LHSarray[:,s][k])
    r2 = [0] * len(indices)
    for i, k in enumerate(indices):
        r2[k] = i

    Ranks2.append(r2)


Ranks3=[]              
indices = list(range(len(subOut)))
indices.sort(key=lambda k: subOut[k])
r3 = [0] * len(indices)
for i, k in enumerate(indices):
    r3[k] = i

Ranks3.append(r3)


Ranks20=[*zip(*Ranks2)]
Ranks30=[*zip(*Ranks3)]
Ranks20array=np.array(Ranks20)
Ranks30array=np.array(Ranks30)
LHSout2 = np.hstack((Ranks20array,Ranks30array))
#LHSout2 = LHSout2.T
SampleResult2 = LHSout2.tolist()
#print(SampleResult2)

Ranks=[]              
for s in range(sampleCount):

    indices = list(range(len(SampleResult[s])))
    indices.sort(key=lambda k: SampleResult[s][k])
    r = [0] * len(indices)
    for i, k in enumerate(indices):
        r[k] = i

    Ranks.append(r)
#print(Ranks)

C= np.corrcoef(SampleResult2, rowvar = False)
#print(C)

if np.linalg.det(C) < 1e-16: # determine if singular
    Cinv = np.linalg.pinv(C) # may need to use pseudo inverse
else:
    Cinv = np.linalg.inv(C) 
    
resultIdx = parameterCount
prcc=np.zeros(resultIdx)

for w in range(parameterCount): # compute PRCC btwn each param & sim result
    prcc[w]=-Cinv[w,resultIdx]/np.sqrt(Cinv[w,w]*Cinv[resultIdx,resultIdx])
    print(prcc)
 


#### Plot the PRCCs for each parameter

#### Guadalupe Station

In [ ]:
xp=[i for i in range(parameterCount)]
print(prcc[0:parameterCount])


plt.bar(xp,prcc[0:parameterCount], align='center',color = 'b')
#plt.bar(xp,prcc1[0:parameterCount], align='center', color = 'r')

bLabels=list(parameters.keys())
plt.xticks(xp, bLabels)
plt.ylim([-1, 1])

plt.ylabel('PRCC value');
plt.xlabel('Parameters');

N=str(sampleCount)
loc=str(x_idx)
plt.title('Partial rank correlation of parameters with TDS results');


plt.savefig('01 January PRCC at at x = 72, t=100.png') 
plt.show()

#### PRCC values over time

In [ ]:
T =  2592000
N = 7200
t = np.linspace(0,T,N+1)

In [ ]:
SampleResult=[]

resultIdx = parameterCount
prcc=np.zeros((resultIdx,len(t)))

LHS=[*zip(*LHSparams)]
LHSarray=np.array(LHS)
Outputarray=np.array(Output)
x_idt = 72

Ranks2=[] 
for s in range(parameterCount):

    indices = list(range(len(LHSarray[:,s])))
    indices.sort(key=lambda k: LHSarray[:,s][k])
    r2 = [0] * len(indices)
    for i, k in enumerate(indices):
        r2[k] = i
        
    Ranks2.append(r2)
    Ranks20=[*zip(*Ranks2)]
    Ranks20array=np.array(Ranks20)


for xi in range(len(t)):  # loop through time or location of sim results 
    xi2  = xi+1           # to compare w/ parameter sample vals

    subOut = Outputarray[0:,x_idt,xi:xi2]
    
    Ranks3=[]              
    indices = list(range(len(subOut)))
    indices.sort(key=lambda k: subOut[k])
    r3 = [0] * len(indices)
    for i, k in enumerate(indices):
        r3[k] = i
        
    Ranks3.append(r3)
    Ranks30=[*zip(*Ranks3)]
    Ranks30array=np.array(Ranks30)
    #print(Ranks3)

    LHSout2 = np.hstack((Ranks20array,Ranks30array))
    SampleResult2 = LHSout2.tolist()
    #print(SampleResult2)
  
    C=np.corrcoef(SampleResult2, rowvar = False);

    if np.linalg.det(C) < 1e-16: # determine if singular
        Cinv = np.linalg.pinv(C)   # may need to use pseudo inverse
    else:
        Cinv = np.linalg.inv(C) 

    for w in range(parameterCount):  # compute PRCC btwn each param & sim result
        prcc[w,xi]=-Cinv[w,resultIdx]/np.sqrt(Cinv[w,w]*Cinv[resultIdx,resultIdx])

#### Plot of PRCC values over time

In [ ]:
for p in range(parameterCount):
    plt.plot(t/360,prcc[p,])
    plt.xlim([0, 1000]) #comment out for the whole duration
    plt.ylim([-1, 1])

labels=list(parameters.keys())
plt.legend(labels)

labelstring = 'TDS'
plt.ylabel('PRCC value');
plt.xlabel('time steps (t)')

N=str(sampleCount)
plt.title('Partial rank correlation of params with ' + labelstring 
          + ' results \n from ' + N + ' LHS sims');

plt.savefig('01 January PRCC at at x = 72 in the time interval [0, 1000].png') 
plt.show()

#### Lambingan Station

In [ ]:
SampleResult=[]

resultIdx = parameterCount+1
prcc=np.zeros((resultIdx,len(t)))

LHS=[*zip(*LHSparams)]
LHSarray=np.array(LHS)
Outputarray=np.array(Output)
x_idt = 170

Ranks2=[] 
for s in range(parameterCount):

    indices = list(range(len(LHSarray[:,s])))
    indices.sort(key=lambda k: LHSarray[:,s][k])
    r2 = [0] * len(indices)
    for i, k in enumerate(indices):
        r2[k] = i
        
    Ranks2.append(r2)
    Ranks20=[*zip(*Ranks2)]
    Ranks20array=np.array(Ranks20)


for xi in range(len(t)):  # loop through time or location of sim results 
    xi2  = xi+1           # to compare w/ parameter sample vals

    subOut = Outputarray[0:,x_idt,xi:xi2]
    
    Ranks3=[]              
    indices = list(range(len(subOut)))
    indices.sort(key=lambda k: subOut[k])
    r3 = [0] * len(indices)
    for i, k in enumerate(indices):
        r3[k] = i
        
    Ranks3.append(r3)
    Ranks30=[*zip(*Ranks3)]
    Ranks30array=np.array(Ranks30)

    LHSout2 = np.hstack((Ranks20array,Ranks30array))
    SampleResult2 = LHSout2.tolist()
    #print(SampleResult2)
  
    C=np.corrcoef(SampleResult2);

    if np.linalg.det(C) < 1e-16: # determine if singular
        Cinv = np.linalg.pinv(C)   # may need to use pseudo inverse
    else:
        Cinv = np.linalg.inv(C) 

    for w in range(parameterCount):  # compute PRCC btwn each param & sim result
        prcc[w,xi]=-Cinv[w,resultIdx]/np.sqrt(Cinv[w,w]*Cinv[resultIdx,resultIdx])

#### Plot of PRCC values for each parameter

In [ ]:
for p in range(parameterCount):
    plt.plot(t/360,prcc[p,])

labels=list(parameters.keys())
plt.legend(labels)

labelstring = 'mao na ni'
plt.ylabel('PRCC value');
plt.xlabel('t')

N=str(sampleCount)
plt.title('Partial rank correlation of params with ' + labelstring 
          + ' results \n from ' + N + ' LHS sims');

plt.show()